## Real experiments for head movements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
import glob

In [4]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [5]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [6]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [7]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [8]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: F2
Training data size: (181593, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [9]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [12]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y',
              'PRIMA:acceleration-r', 'PRIMA:acceleration-clock', 'PRIMA:acceleration-x',
              'PRIMA:acceleration-y', 'PRIMA:jerk-r', 'PRIMA:jerk-clock', 'PRIMA:jerk-x', 'PRIMA:jerk-y',
              'PRIMA:pitch', 'PRIMA:intensity',
              'PRIMA:word01'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
    #Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
#    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0]
 [ 4259     0]]
0.7405105708889295
0.42545594567157907
0.6301092504347008

Trying classifier  LR F5
[[11435   719]
 [ 3094  1165]]
0.7676841528057028
0.6181967655180728
0.7331142847620689

Trying classifier  LINEARSVC F5
[[11522   632]
 [ 3162  1097]]
0.7688417717662829
0.6125163733831773
0.7309038093769057

Trying classifier  MLP F5
[[10918  1236]
 [ 2711  1548]]
0.7595198927679279
0.6432500188102661
0.7412170043914549

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Trying classifier  MF M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[9316    0]
 [7216    0]]
0.5635131865473022
0.36041473228102755
0.4061969085325493

Trying classifier  LR M6
[[8339  977]
 [4705 2511]]
0.6563029276554561
0.6075279567132211
0.6251030148770326

Trying classifier  LINEARSVC M6
[[8406  910]
 [4857 2359]]
0.6511613839825793
0.5972809166851942
0.6159925061220238

Trying classifier  MLP M6
[[8048 1268]
 [3931 3285]]
0.6855189934672151
0.6570522113971721
0.6696031755683017

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Trying classifier  MF M4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10315     0]
 [ 6152     0]]
0.6264043237991134
0.38514674034799495
0.48251516690223695

Trying classifier  LR F2
[[8028 2287]
 [2084 4068]]
0.7345600291492075
0.7182671080608429
0.7353952470736869

Trying classifier  LINEARSVC F2
[[8197 2118]
 [2200 3952]]
0.7377785874779863
0.7191122452094726
0.7374180126685591

Trying classifier  MLP F2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[7413 2902]
 [1686 4466]]
0.7213821582559058
0.7121632927619488
0.725186074252055



,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.767684,0.656303,0.712270,0.674203,0.753424,0.672908,0.744549,0.766021,0.769841,0.700551,0.752493,0.734560
LINEARSVC,0.768842,0.651161,0.709242,0.668231,0.752980,0.667050,0.740117,0.766683,0.767363,0.695695,0.751515,0.737779
MLP,0.759520,0.685519,0.728379,0.698032,0.759024,0.700649,0.748153,0.743838,0.770027,0.695432,0.751320,0.721382


In [13]:
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.767684,0.656303,0.712270,0.674203,0.753424,0.672908,0.744549,0.766021,0.769841,0.700551,0.752493,0.734560
LINEARSVC,0.768842,0.651161,0.709242,0.668231,0.752980,0.667050,0.740117,0.766683,0.767363,0.695695,0.751515,0.737779
MLP,0.759520,0.685519,0.728379,0.698032,0.759024,0.700649,0.748153,0.743838,0.770027,0.695432,0.751320,0.721382


In [14]:
f1score_resultsw

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.630109,0.406197,0.509992,0.436293,0.567316,0.391594,0.439536,0.602434,0.539715,0.340577,0.595607,0.482515
LR,0.733114,0.625103,0.686093,0.631731,0.740462,0.631452,0.731947,0.754203,0.752809,0.698953,0.710569,0.735395
LINEARSVC,0.730904,0.615993,0.678625,0.620797,0.736826,0.621009,0.724842,0.751262,0.747074,0.693099,0.705650,0.737418
MLP,0.741217,0.669603,0.717726,0.675693,0.751733,0.677319,0.743964,0.747618,0.762403,0.695155,0.732347,0.725186


In [15]:
f1score_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.425456,0.360415,0.393320,0.370555,0.409328,0.355291,0.371616,0.418514,0.401786,0.336208,0.416762,0.385147
LR,0.618197,0.607528,0.637100,0.601546,0.682584,0.616410,0.715255,0.682804,0.705598,0.698642,0.608851,0.718267
LINEARSVC,0.612516,0.597281,0.626465,0.588716,0.675745,0.605015,0.706769,0.675859,0.696911,0.692705,0.600010,0.719112
MLP,0.643250,0.657052,0.681463,0.653717,0.700779,0.666396,0.731898,0.692239,0.723435,0.695252,0.653172,0.712163


In [16]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.725401,0.723055,0.730106
std,0.075262,0.040814,0.043093,0.029272
min,0.506497,0.656303,0.651161,0.685519
25%,0.582404,0.693964,0.688829,0.699995
50%,0.637360,0.739554,0.738948,0.736109
75%,0.698381,0.756573,0.756406,0.753246
max,0.740511,0.769841,0.768842,0.770027


In [17]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.387033,0.657732,0.649759,0.684235
std,0.028472,0.046659,0.048963,0.029678
min,0.336208,0.601546,0.588716,0.643250
25%,0.368020,0.614520,0.603764,0.656219
50%,0.389234,0.659842,0.651105,0.686851
75%,0.411186,0.700381,0.693757,0.703625
max,0.425456,0.718267,0.719112,0.731898
